<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#DPA-Attack-Theory" data-toc-modified-id="DPA-Attack-Theory-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>DPA Attack Theory</a></span></li><li><span><a href="#Capturing-Power-Traces" data-toc-modified-id="Capturing-Power-Traces-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Capturing Power Traces</a></span><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#Capture" data-toc-modified-id="Capture-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Capture</a></span></li></ul></li><li><span><a href="#Analysis" data-toc-modified-id="Analysis-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Analysis</a></span></li><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Conclusion</a></span></li><li><span><a href="#Tests" data-toc-modified-id="Tests-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Tests</a></span></li></ul></div>

# PA_DPA_3-AES_DPA_Attack

Before starting this tutorial, it's recommended that you first complete the earlier PA_DPA tutorials since these will familiarize you with the concept of Differental Power Analysis. With that out of the way, let's look at how this attack works.

## DPA Attack Theory

As we explored in the earlier DPA tutorials, the Hamming Weight of the result of the SBox operation in AES has a measurable effect on the power consumed by the microcontroller. It turns out that just this effect (and not anything stronger, such as its linearity) is enough information to break an AES key. There's a few different ways we could go about this, but for this tutorial, we'll be looking at difference of means. With this technique, the goal is to separate the traces by a bit in the result of the SBox output (it doesn't matter which one): if that bit is 1, its group of traces should, on average, have higher power consumption during the SBox operation than the other set. 

Whether or not we get a large difference in the means between these two groups depends on whether they were properly sorted into these groups. If not, there should be, on average, little difference between the two and therefore a low difference of means. Recall the SBox operation:

![title](https://wiki.newae.com/images/7/71/Sbox_cpa_detail.png)

The SBox output depends on the subkey, which we don't know (and the plaintext, which we do). However, since there's a large difference of means for the correct key and small ones for the rest of the possible subkeys, we have a method of checking whether a given subkey is correct. If we calculate the difference of means for each subkey, the correct one will have the largest difference of means.

Our plan looks as follows
1. Capture a bunch of power traces with varying plaintext
1. Group each trace by the value of their SBox output's lowest bit for a given key guess
1. Calculate the difference of means
1. Repeat for each possible subkey
1. Select the largest difference of means -> this is the correct subkey
1. Repeat for each subkey in the key

At the end, we should get a correct AES key!

In [1]:
SCOPETYPE = "OPENADC"
PLATFORM = "CWLITEXMEGA"
CRYPTO_TARGET="AVRCRYPTOLIB"

In [2]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../hardware/victims/firmware/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2

rm -f -- simpleserial-aes-CWLITEXMEGA.hex
rm -f -- simpleserial-aes-CWLITEXMEGA.eep
rm -f -- simpleserial-aes-CWLITEXMEGA.cof
rm -f -- simpleserial-aes-CWLITEXMEGA.elf
rm -f -- simpleserial-aes-CWLITEXMEGA.map
rm -f -- simpleserial-aes-CWLITEXMEGA.sym
rm -f -- simpleserial-aes-CWLITEXMEGA.lss
rm -f -- objdir/*.o
rm -f -- objdir/*.lst
rm -f -- simpleserial-aes.s simpleserial.s XMEGA_AES_driver.s uart.s usart_driver.s xmega_hal.s aes-independant.s aes_enc.s aes_keyschedule.s aes_sbox.s aes128_enc.s
rm -f -- simpleserial-aes.d simpleserial.d XMEGA_AES_driver.d uart.d usart_driver.d xmega_hal.d aes-independant.d aes_enc.d aes_keyschedule.d aes_sbox.d aes128_enc.d
rm -f -- simpleserial-aes.i simpleserial.i XMEGA_AES_driver.i uart.i usart_driver.i xmega_hal.i aes-independant.i aes_enc.i aes_keyschedule.i aes_sbox.i aes128_enc.i
.
-------- begin --------
avr-gcc (WinAVR 20100110) 4.3.3
Copyright (C) 2008 Free Software Foundation, Inc.
This is free software; see the source for copying conditio

## Capturing Power Traces

Capture and setup is similar to earlier tutorials. We'll have to capture a fair number of traces (usually a few thousand here) since Difference of Means isn't a super trace efficient method. As you'll find during the CPA tutorials, CPA is much better in this regard - it can often break AES implementations such as these in under 50 traces.

You may also find that you need to modify gain settings and the number of traces you capture - this attack is much more sensitive to gain settings and noise than a CPA attack would be. 

### Setup

In [3]:
%run "Helper_Scripts/Setup.ipynb"

In [4]:
fw_path = "../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex".format(PLATFORM)

In [5]:
cw.programTarget(scope, prog, fw_path)

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 3471 bytes


### Capture

In [13]:
#Capture Traces
from tqdm import tnrange
import numpy as np
import time

ktp = cw.ktp.Basic(target=target)

traces = []
textin = []
keys = []
N = 2000  # Number of traces

if PLATFORM == "CWLITEARM" or PLATFORM == "CW308_STM32F3":
    scope.adc.samples = 4000
elif PLATFORM == "CWLITEXMEGA" or PLATFORM == "CW303":
    scope.gain.db = 34 #works best with this gain for some reason
    scope.adc.samples = 1700 - 170
    scope.adc.offset = 500 + 700 + 170
    N = 5000
target.init()
print(scope)
for i in tnrange(N, desc='Capturing traces'):
    # run aux stuff that should come before trace here

    key, text = ktp.newPair()  # manual creation of a key, text pair can be substituted here
    textin.append(text)
    keys.append(key)
    
    target.loadEncryptionKey(key)
    target.loadInput(text)

    # run aux stuff that should run before the scope arms here

    scope.arm()

    # run aux stuff that should run after the scope arms here

    target.go()
    timeout = 50
    # wait for target to finish
    while target.isDone() is False and timeout:
        timeout -= 1
        time.sleep(0.01)

    try:
        ret = scope.capture()
        if ret:
            print('Timeout happened during acquisition')
    except IOError as e:
        print('IOError: %s' % str(e))

    # run aux stuff that should happen after trace here
    _ = target.readOutput()  # clears the response from the serial port
    traces.append(scope.getLastTrace())

#Convert traces to numpy arrays
trace_array = np.asarray(traces)  # if you prefer to work with numpy array for number crunching
textin_array = np.asarray(textin)
known_keys = np.asarray(keys)  # for fixed key, these keys are all the same

cwlite Device
gain = 
    mode = high
    gain = 44
    db   = 33.859375
adc = 
    state      = False
    basic_mode = rising_edge
    timeout    = 2
    offset     = 1370
    presamples = 0
    samples    = 1530
    decimate   = 1
    trig_count = 69824
clock = 
    adc_src       = clkgen_x4
    adc_phase     = 0
    adc_freq      = 29538459
    adc_rate      = 29538459.0
    adc_locked    = True
    freq_ctr      = 0
    freq_ctr_src  = extclk
    clkgen_src    = system
    extclk_freq   = 10000000
    clkgen_mul    = 2
    clkgen_div    = 26
    clkgen_freq   = 7384615.384615385
    clkgen_locked = True
trigger = 
    triggers = tio4
    module   = basic
io = 
    tio1       = serial_rx
    tio2       = serial_tx
    tio3       = high_z
    tio4       = high_z
    pdid       = high_z
    pdic       = high_z
    nrst       = high_z
    glitch_hp  = False
    glitch_lp  = False
    extclk_src = hs1
    hs2        = clkgen
    target_pwr = True
glitch = 
    clk_src     = target
    w

## Analysis

As we discussed above, our goal here is to find the biggest difference of means out of the possible subkey values we could have. First, we'll get some values and functions that will be useful for our calculations. We'll be using `intermediate()` later to get the output of the SBox from a plaintext and key input.

In [14]:
numtraces = np.shape(trace_array)[0] #total number of traces
numpoints = np.shape(trace_array)[1] #samples per trace

sbox = (
    0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76,
    0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0,
    0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15,
    0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75,
    0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84,
    0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf,
    0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8,
    0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2,
    0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73,
    0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb,
    0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
    0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08,
    0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a,
    0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e,
    0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf,
    0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16)

def intermediate(pt, keyguess):
    return sbox[pt ^ keyguess]

Our first step here will be separating our traces into different groups based on the SBox's output. As mentioned earlier, we're separating based on the least significant bit, but really any bit would work (as a test, you can change this and see if the attack still works):

```Python
one_list = []
zero_list = []
for tnum in range(numtraces):
    if (intermediate(textin_array[tnum][subkey], kguess) & 1):
        one_list.append(trace_array[tnum])
    else:
        zero_list.append(trace_array[tnum])
```

Then calculate the difference of means:

```Python
one_avg = np.asarray(one_list).mean(axis=0)
zero_avg = np.asarray(zero_list).mean(axis=0)
mean_diffs[kguess] = np.max(abs(one_avg - zero_avg))
```

We'll need to repeat this with each possible key guess and then pick the one with the highest difference of means:
```Python
guess = np.argsort(mean_diffs)[-1]
key_guess.append(guess)
print(hex(guess))
print(mean_diffs[guess])
```

Finally, altogether and attacking all of the subkeys:

In [15]:
from tqdm import tnrange
import numpy as np
mean_diffs = np.zeros(255)
key_guess = []
known_key = known_keys[0]
plots = []
for subkey in tnrange(0, 16, desc="Attacking Subkey"):
    for kguess in tnrange(255, desc="Keyguess", leave=False):
        one_list = []
        zero_list = []
        
        for tnum in range(numtraces):
            if (intermediate(textin_array[tnum][subkey], kguess) & 1): #LSB is 1
                one_list.append(trace_array[tnum])
            else:
                zero_list.append(trace_array[tnum])
        one_avg = np.asarray(one_list).mean(axis=0)
        zero_avg = np.asarray(zero_list).mean(axis=0)
        mean_diffs[kguess] = np.max(abs(one_avg - zero_avg))
        if kguess == known_key[subkey]:
            plots.append(abs(one_avg - zero_avg))
    guess = np.argsort(mean_diffs)[-1]
    key_guess.append(guess)
    print(hex(guess) + "(real = 0x{:02X})".format(known_key[subkey]))
    #mean_diffs.sort()
    print(mean_diffs[guess])
    print(mean_diffs[known_key[subkey]])

0x2b(real = 0x2B)
0.015823563752175196
0.015823563752175196


0x7e(real = 0x7E)
0.018061103494413977
0.018061103494413977


0x15(real = 0x15)
0.016951614681458727
0.016951614681458727


0x16(real = 0x16)
0.015873129883450743
0.015873129883450743


0x28(real = 0x28)
0.016513993816346106
0.016513993816346106


...

0x88(real = 0x88)
0.015350254940920904
0.015350254940920904


0x9(real = 0x09)
0.015979218194920042
0.015979218194920042


0xcf(real = 0xCF)
0.0163163905199597
0.0163163905199597


0x4f(real = 0x4F)
0.015904434332774386
0.015904434332774386


0x3c(real = 0x3C)
0.017139865436179402
0.017139865436179402



With that done, we should now have the correct key:

In [16]:
print(key_guess)
print(known_key)

[43, 126, 21, 22, 40, 174, 210, 166, 171, 247, 21, 136, 9, 207, 79, 60]
[ 43 126  21  22  40 174 210 166 171 247  21 136   9 207  79  60]


We can also plot the difference of means for a few of the correct subkey bytes:

In [17]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()
p = figure()
p.line(range(numpoints), plots[0], line_color='green')
p.line(range(numpoints), plots[1], line_color='red')
p.line(range(numpoints), plots[15], line_color='blue')
show(p)

Loading BokehJS ...

## Conclusion

Congratulations, you have (hopefully) broken AES using a DPA attack! As you might have discovered during this tutorial, there can be quite a few issues with the difference of means method for breaking AES keys:

* It's quite susceptible to noise
* The attack can easily pick up other parts of the AES operation
* The attack typically requires a lot of traces. These software AES implementations are pretty weak against power analysis, but they still required thousands of traces to break
* Some targets (such as the XMega) require fine tuning settings to make the attack work

Nevertheless, using a difference of means attack can still be very useful. For example, a later tutorial, PA_Multi_1, uses a difference of means attack similar in concept to this one to break the signature of an AES256 bootloader.

## Tests

In [18]:
assert (known_key == key_guess).all(), "Failed to break key.\nGot: {}\nExp: {}".format(key_guess, known_key)